# Import packages

In [1]:
import pandas as pd
from teradataml import *
import ast
import json


from utils import clean_text
from constants import (

SHOPMAINA_DATASET, SKROUTZ_DATASET, PRICE_RUNNER_DATASET, JIO_MART_DATASET, JIO_MART_DATASET_MAPPED, JIO_MART_DATASET_MAPPING
)


In [2]:
jio_mart_df = pd.read_csv(JIO_MART_DATASET)
jio_mart_df = jio_mart_df.loc[~jio_mart_df['items'].isna(), :]
jio_mart_df = jio_mart_df.drop(columns=['href', 'price'])
jio_mart_df.rename(columns={'items': 'product name'}, inplace=True)
jio_mart_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162280 entries, 0 to 162312
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   category      162280 non-null  object
 1   sub_category  162280 non-null  object
 2   product name  162280 non-null  object
dtypes: object(3)
memory usage: 5.0+ MB


In [3]:
jio_mart_df.head()

,category,sub_category,product name
0,Groceries,Fruits & Vegetables,Fresh Dates (Pack) (Approx 450 g - 500 g)
1,Groceries,Fruits & Vegetables,Tender Coconut Cling Wrapped (1 pc) (Approx 90...
2,Groceries,Fruits & Vegetables,Mosambi 1 kg
3,Groceries,Fruits & Vegetables,Orange Imported 1 kg
4,Groceries,Fruits & Vegetables,Banana Robusta 6 pcs (Box) (Approx 800 g - 110...


In [13]:
jio_mart_df_cateogires = jio_mart_df["sub_category"].unique()
jio_mart_df_cateogires

array(['Fruits & Vegetables', 'Premium Fruits', 'Dairy & Bakery',
       'Staples', 'Snacks & Branded Foods', 'Beverages', 'Personal Care',
       'Home Care', 'Apparel', 'Mom & Baby Care', 'Books', 'Pets',
       'Kitchenware', 'Dining', 'Furnishing', 'Home Decor', 'Furniture',
       'Home Appliances', 'Toys, Games & Fitness', 'Electrical',
       'Bathroom & Laundry Accessories', 'Disposables', 'Stationery',
       'Bags & Travel Luggage', 'Mops, Brushes & Scrubs', 'Auto Care',
       'Garden & Outdoor', 'Carpentry & work accessories', 'Pooja Needs',
       'Bathroom & Laundry', 'Industrial & Scientific Supplies',
       'Building Supplies & Measuring Tools', 'Hardware & Plumbing',
       'Home Safety & Automation', 'Kitchen & Bath Fixtures',
       'Paint, Wall Treatments & Supplies', 'Power & Hand Tools',
       'Handloom & Handicraft', 'Personal Wear', 'Men', 'Women', 'Boys',
       'Girls', 'Junior Boys', 'Infants', 'Mobiles & Tablets',
       'TV & Speaker', 'Computers', 'Camer

In [4]:
with open(JIO_MART_DATASET_MAPPING, "r", encoding="utf-8") as f:
    mapping = json.load(f)

In [5]:
def map_to_gpc(category, subcategory):
    try:
        return mapping[category][subcategory]
    except KeyError:
        return [None, None, None, None]

In [7]:
jio_mart_df[["Segment", "Family", "Class", "Brick"]] = jio_mart_df.apply(
    lambda row: pd.Series(map_to_gpc(row["category"], row["sub_category"])),
    axis=1
)

In [11]:
jio_mart_df.to_csv(JIO_MART_DATASET_MAPPED, index=False)